In [5]:
import os, socket

socket.gethostname()

'DESKTOP-0IJAC10'

In [6]:
pwd

'c:\\Users\\dytpq0916\\VTO2025\\REFERENCES\\SewFormerAnalysis\\Sewformer\\ANALYSIS\\WINPC'

 'C:\Program Files\Autodesk\Maya2022\bin\mayapy.exe' -m IPython

 & 'C:\Program Files\Autodesk\Maya2022\bin\mayapy.exe'

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from glob import glob
import math
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import json
import random
import pandas as pd
from tqdm import tqdm
import trimesh
import pickle
import pandas as pd
# import torch

from ANALYSIS.analysis_utils import (
    plot_panel_info,
    visualize_meshes_plotly,
    filter_segmentation_map,
    filter_segmentation_map_clusters,
    is_clockwise,
)

from env_constants import SEWFORMER_PROJ_ROOT, DATASET_ROOT, PYGARMENT_ROOT

sys.path.append(PYGARMENT_ROOT)

import pygarment as pyg

c:\Users\dytpq0916\VTO2025\REFERENCES\SewFormerAnalysis\Sewformer\env_constants.py:24: SyntaxWarning: invalid escape sequence '\V'
  DATASET_ROOT        = "D:\VTO2025\sewfactory"
C:\Users\dytpq0916\VTO2025\REFERENCES\GarmentCodeAnalysis\pygarment\garmentcode\utils.py:83: SyntaxWarning: invalid escape sequence '\i'
  raise ValueError(f'lin_interpolation::ERROR::Expected a factor \in [0, 1], got {factor}')
C:\Users\dytpq0916\VTO2025\REFERENCES\GarmentCodeAnalysis\pygarment\pattern\core.py:636: SyntaxWarning: invalid escape sequence '\c'
  """


In [6]:
garment_df = pd.read_csv("garment_df_20250215_165422.csv")

filtered_combination_name_list = np.unique(garment_df[
    garment_df["mesh_filter_failed"] == False
]["matching_combination_name"]).tolist()

filtered_combination_path_list = list(map(
    lambda x : os.path.join(DATASET_ROOT, 'sewfactory', x),
    filtered_combination_name_list
))

In [7]:
joints_name = (
    'Pelvis', 'L_Hip', 'R_Hip', 'Torso', 'L_Knee', 'R_Knee',
    'Spine', 'L_Ankle', 'R_Ankle', 'Chest', 'L_Toe', 'R_Toe',
    'Neck', 'L_Thorax', 'R_Thorax', 'Head', 'L_Shoulder', 'R_Shoulder',
    'L_Elbow', 'R_Elbow', 'L_Wrist', 'R_Wrist', 'L_Hand', 'R_Hand', 'Nose',
    'L_Eye', 'R_Eye', 'L_Ear', 'R_Ear', 'Head_top'
)
skeleton = (
    (0,1), (1,4), (4,7), (7,10), (0,2), (2,5), (5,8), (8,11), (0,3), (3,6), (6,9),
    (9,14), (14,17), (17,19), (19, 21), (21,23), (9,13), (13,16), (16,18), (18,20),
    (20,22), (9,12), (12,24), (24,15), (24,25), (24,26), (25,27), (26,28), (24,29)
)

In [ ]:
def amass_to_pose(src_path, save_path,save_path1):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    
    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    vertex_seq = []
    if bdata['gender'] == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)
    root_data=0
    with torch.no_grad():
        for fId in range(0, frame_number, down_sample):
            root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
            #tt_test=root_orient[0][2].detach().clone()
            if fId==0:
                root_data=root_orient
            root_orient=root_orient-root_data
            #root_orient[0][1]=tt_test
            pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
            pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
            betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape

            #trans = torch.Tensor(bdata['trans'][fId:fId+1]).to(comp_device)    
            body = bm(pose_body=pose_body, pose_hand=pose_hand, betas=betas, root_orient=root_orient)
            joint_loc = body.Jtr[0] #+ trans
            vertex=body.v[0]
            pose_seq.append(joint_loc.unsqueeze(0))
            vertex_seq.append(vertex.unsqueeze(0))
    pose_seq = torch.cat(pose_seq, dim=0)
    vertex_seq = torch.cat(vertex_seq, dim=0).detach().cpu().numpy()
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    np.save(save_path, pose_seq_np)
    np.save(save_path1,vertex_seq)
    return fps

In [13]:
body_info_path = os.path.join(
    SEWFORMER_PROJ_ROOT,
    "SewFactory", "examples", "fuckingHell__body_info.json"
)

body_info = json.load(open(body_info_path, "r"))

body_info.keys()

np.array(body_info["shape"]).shape, np.array(body_info["trans"]).shape, np.array(body_info["pose"]).shape


((10,), (3,), (52, 3))

In [14]:
np.array(body_info["pose"])


array([[ 0.76705712,  1.66349971,  1.84736717],
       [-0.13965544, -0.08009317, -0.04988057],
       [ 0.25977683,  0.03148104, -0.0783634 ],
       [-0.19558825,  0.0770316 ,  0.08279401],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.00602317,  0.02500766, -0.14073037],
       [ 0.98920232, -0.10086214, -0.06216767],
       [ 0.21144934, -0.15072259,  0.16301364],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.09141605,  0.07704409, -0.09555406],
       [ 0.11973252, -0.01931608,  0.03081282],
       [-0.03203873,  0.04941078,  0.00402705],
       [-0.10786904,  0.05210842,  0.10010238],
       [ 0.02678382, -0.05839427, -0.04454228],
       [ 0.05099276, -0.06246081, -0.34794655],
       [ 0.15228388, -0.16913849, -1.12803459],
       [ 0.26461682, -0.44811746,  0.27054751],
       [-0.32592213, -0.26719871, -0.17984928],
       [-0.07072913,  0.2990959 ,  0.39636588],
       [-0.20578609,  0.64986241,  0.85632759],
       [-0.66297793,  0.27196911, -0.394

In [5]:
pose_file_path = os.path.join(
    SEWFORMER_PROJ_ROOT,
    "SewFactory", "examples", "human_poses",
    "walkdog_poses.npz"
)

data = np.load(pose_file_path)
if 'poses' in data.keys():
    poses = data['poses']
    N = poses.shape[0]
    cdata_ids = list(range(int(0.1*N), int(0.9*N),1))
    poses = data['poses'][cdata_ids].astype(np.float32)
    trans = data['trans'][cdata_ids].astype(np.float32)
    total_frames = poses.shape[0]
gender = data['gender']

In [9]:
poses.shape, trans.shape, total_frames, gender

((1564, 156), (1564, 3), 1564, array('male', dtype='<U4'))

In [2]:
SEWFORMER_PROJ_ROOT

'c:\\Users\\dytpq0916\\VTO2025\\REFERENCES\\SewFormerAnalysis'

In [23]:
IDX = 0
poses[3].reshape(-1, 3) == np.array(body_info["pose"])

for row1, row2 in zip(poses[3].reshape(-1, 3), np.array(body_info["pose"])):
    print(f"{row1 == row2}  {row1}            {row2}")







[ True  True False]  [0.7670571 1.6634997 1.8473672]            [0.76705712 1.66349971 1.84736717]
[ True  True  True]  [-0.13965544 -0.08009317 -0.04988057]            [-0.13965544 -0.08009317 -0.04988057]
[False False False]  [ 0.00602317  0.02500766 -0.14073037]            [ 0.25977683  0.03148104 -0.0783634 ]
[False False False]  [ 0.09141605  0.07704409 -0.09555406]            [-0.19558825  0.0770316   0.08279401]
[False False False]  [ 0.25977683  0.03148104 -0.0783634 ]            [0. 0. 0.]
[False False False]  [ 0.9892023  -0.10086214 -0.06216767]            [ 0.00602317  0.02500766 -0.14073037]
[False False False]  [ 0.11973252 -0.01931608  0.03081282]            [ 0.98920232 -0.10086214 -0.06216767]
[False False False]  [-0.19558825  0.0770316   0.08279401]            [ 0.21144934 -0.15072259  0.16301364]
[False False False]  [ 0.21144934 -0.1507226   0.16301364]            [0. 0. 0.]
[False False False]  [-0.03203873  0.04941078  0.00402705]            [ 0.09141605  0.07704